# Battle of the Neighborhoods Exploration


## Table of Contents

1.  <a href="#item1">Foursquare API Search Function</a>
2.  <a href="#item2">Explore a Given Venue</a>  
3.  <a href="#item3">Explore a User</a>  
4.  <a href="#item4">Foursquare API Explore Function</a>  
5.  <a href="#item5">Get Trending Venues</a>  


### Import necessary Libraries


In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


In [87]:
CLIENT_ID = 'SOJPKPZ015OYSMWNI33FDE45WPXV1TYNA41FTCGFHSNFPNRL' # your Foursquare ID
CLIENT_SECRET = 'UN4FY2DCKDANXOWXMDJOJWQR530BWZI3WVY2WYIMTUZQTOZX' # your Foursquare Secret
CODE = 'MYYSTTB2KGVIN4UJ34I0ZT0ZYHQBXVU3TZIIDT3STVXXSYPH#_=_'
ACCESS_TOKEN = 'IEWY5KTKBXWCB5VHGGC5ZGIRZ2F4BN3MUTNXOAWPW0RLA0GA' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.


In [115]:
address = 'Berkeley, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.8708393 -122.2728639


In [116]:
latitude = 37.871794
longitude = -122.259988

In [117]:
search_query = 'Coffee'
# radius in meters
radius = 2000

In [118]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [119]:
results = requests.get(url).json()

In [120]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

#### Define information of interest and filter dataframe


In [125]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head(1)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Peet's Coffee & Tea,Coffee Shop,"2501 Telegraph Ave.,",at Dwight Way,37.865053,-122.258319,"[{'label': 'display', 'lat': 37.865053, 'lng':...",764,94704,US,Berkeley,CA,United States,"[2501 Telegraph Ave., (at Dwight Way), Berkele...",NaN,4af355c3f964a520b4ec21e3


#### Let's visualize the Italian restaurants that are nearby


In [122]:
dataframe_filtered.name

0               Peet's Coffee & Tea
1            Coffee Bean & Tea Leaf
2               Peet's Coffee & Tea
3                    Tully's Coffee
4               Peet's Coffee & Tea
5        Royal Grounds Coffee House
6               Peet's Coffee & Tea
7       Das Is Good Coffee Roasters
8                        Coffee Lab
9               Peet's Coffee & Tea
10               Blue Bottle Coffee
11                   WFM Coffee Bar
12       Sodoi Coffee Tasting House
13                   Romeo's Coffee
14         Cafenated Coffee Company
15              1951 Coffee Company
16    Wrecking Ball Coffee Roasters
17              1951 Coffee Company
18                    Remedy Coffee
19                  The Coffee Club
20                     SoDoI Coffee
21               My Coffee Roastery
22                    Modern Coffee
23              Pipeline Coffee Co.
24            Windmill Coffee House
25                 Algorithm Coffee
26         Scooters Coffee & Yogurt
27                   Seismic

In [123]:
dataframe_filtered.categories

0     Coffee Shop
1     Coffee Shop
2     Coffee Shop
3     Coffee Shop
4     Coffee Shop
5     Coffee Shop
6     Coffee Shop
7     Coffee Shop
8     Coffee Shop
9     Coffee Shop
10    Coffee Shop
11    Coffee Shop
12    Coffee Shop
13           Café
14    Coffee Shop
15           Café
16    Coffee Shop
17           Café
18    Coffee Shop
19    Coffee Shop
20    Coffee Shop
21    Coffee Shop
22    Coffee Shop
23    Coffee Shop
24    Coffee Shop
25    Coffee Shop
26    Coffee Shop
27    Coffee Shop
28           Café
29           Food
Name: categories, dtype: object

In [124]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Berkeley, CA',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_


In [98]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=SOJPKPZ015OYSMWNI33FDE45WPXV1TYNA41FTCGFHSNFPNRL&client_secret=UN4FY2DCKDANXOWXMDJOJWQR530BWZI3WVY2WYIMTUZQTOZX&oauth_token=IEWY5KTKBXWCB5VHGGC5ZGIRZ2F4BN3MUTNXOAWPW0RLA0GA&v=20180604'

#### Send GET request for result


In [99]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St', 'New York, NY 10282']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaura

### B. Get the venue's overall rating


In [100]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.9


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.


In [101]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.


In [102]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


Since this restaurant has a slightly better rating, let's explore it further.


## 4. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.


In [103]:
latitude = 37.871794
longitude = -122.259988

#### Define URL


In [104]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SOJPKPZ015OYSMWNI33FDE45WPXV1TYNA41FTCGFHSNFPNRL&client_secret=UN4FY2DCKDANXOWXMDJOJWQR530BWZI3WVY2WYIMTUZQTOZX&ll=37.871794,-122.259988&v=20180604&radius=2000&limit=30'

#### Send GET request and examine results


In [105]:
import requests

In [106]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

#### Get relevant part of JSON


In [107]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ba106c4f964a520579037e3',
  'name': 'Campanile (Sather Tower)',
  'location': {'address': 'Campanile Esplanade',
   'lat': 37.87193214890736,
   'lng': -122.2580968508397,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.87193214890736,
     'lng': -122.2580968508397}],
   'distance': 166,
   'postalCode': '94709',
   'cc': 'US',
   'city': 'Berkeley',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['Campanile Esplanade',
    'Berkeley, CA 94709',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d165941735',
    'name': 'Scenic Lookout',
    'pluralName': 'Scenic Lookouts',
    'shortName': 'Scenic Lookout',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/sceniclookout_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups

#### Process JSON and convert it to a clean dataframe


In [108]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Campanile (Sather Tower),Scenic Lookout,Campanile Esplanade,37.871932,-122.258097,"[{'label': 'display', 'lat': 37.87193214890736...",166,94709,US,Berkeley,CA,United States,"[Campanile Esplanade, Berkeley, CA 94709, Unit...",NaN,NaN,4ba106c4f964a520579037e3
1,Zellerbach Hall,College Theater,101 Zellerbach Hall,37.869167,-122.260498,"[{'label': 'display', 'lat': 37.86916713570641...",295,94720,US,Berkeley,CA,United States,"[101 Zellerbach Hall (at Telegraph Ave.), Berk...",at Telegraph Ave.,NaN,44169375f964a52006311fe3
2,Woo Hon Fai Hall,Performing Arts Venue,2625 Durant Ave,37.868679,-122.255867,"[{'label': 'display', 'lat': 37.86867904663086...",501,94720,US,Berkeley,CA,United States,"[2625 Durant Ave, Berkeley, CA 94720, United S...",NaN,NaN,590d8f5d018cbb785f3b6578
3,Hertz Concert Hall,Concert Hall,U C Berkeley,37.870978,-122.255775,"[{'label': 'display', 'lat': 37.87097799864652...",381,94720,US,Berkeley,CA,United States,"[U C Berkeley (at UC Berkeley), Berkeley, CA 9...",at UC Berkeley,NaN,4aea4877f964a520bcba21e3
4,William Randolph Hearst Greek Theatre,Amphitheater,2001 Gayley Rd,37.873748,-122.254245,"[{'label': 'display', 'lat': 37.87374820090017...",549,94720,US,Berkeley,CA,United States,"[2001 Gayley Rd, Berkeley, CA 94720, United St...",NaN,NaN,4c0082bd34ccc928bec5e1cd
5,Cinnaholic,Bakery,2132 Oxford Street,37.870304,-122.266084,"[{'label': 'entrance', 'lat': 37.870258, 'lng'...",560,94704,US,Berkeley,CA,United States,"[2132 Oxford Street (at Center St), Berkeley, ...",at Center St,NaN,4c40b646d691c9b6c0a08b0a
6,BAMPFA,Art Museum,2155 Center St,37.870856,-122.266455,"[{'label': 'display', 'lat': 37.87085586060765...",577,94704,US,Berkeley,CA,United States,"[2155 Center St (at Oxford St), Berkeley, CA 9...",at Oxford St,NaN,56aad9ff498e06046ff5e01b
7,Top Dog,Hot Dog Joint,2534 Durant Ave,37.867986,-122.257397,"[{'label': 'display', 'lat': 37.86798622781440...",481,94704,US,Berkeley,CA,United States,[2534 Durant Ave (btwn Telegraph & Bowditch St...,btwn Telegraph & Bowditch St,NaN,4a7414a6f964a520b8dd1fe3
8,Amazon@ASUC Student Union,Shipping Store,MLK Student Union 2495 Bancroft Way Room 235,37.869297,-122.259387,"[{'label': 'display', 'lat': 37.86929705907425...",282,94720,US,Berkeley,CA,United States,"[MLK Student Union 2495 Bancroft Way Room 235,...",NaN,NaN,56c60eadcd10fb359a0a1fa6
9,The pool at Berkeley City Club,Pool,2315 Durant Ave,37.867612,-122.262543,"[{'label': 'display', 'lat': 37.86761217476545...",516,94704,US,Berkeley,CA,United States,"[2315 Durant Ave, Berkeley, CA 94704, United S...",NaN,NaN,5aa39ae5838e59757f4509c3


#### Let's visualize these items on the map around our location


In [109]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>


## 5. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.


In [110]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601dbc1311e9d93e69eeed69'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [111]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [112]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [113]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [114]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'